In [ ]:
import pandas as pd 
import numpy as np
from datetime import timedelta
import missingno as msno
from scipy import stats
import statistics
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px

In [ ]:
df=pd.read_csv(r'../input/covid19-case-surveillance-public-use-dataset/COVID-19_Case_Surveillance_Public_Use_Data.csv')


In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
print("The number of columns present in a dataset are:",df.shape[1])
print("The number of rows present in a dataset are:",df.shape[0])

In [ ]:
df.columns

# Column Description

1.cdc_report_dt - earlier clinical dates related to illness or specimen collection

2.pos_spec_dt   - date at which column name was submitted to the database

3.onset_dt      - date of the symptoms shown

4.sex           - gender of a person

5.age_group     - various age groups of a person

6.Race and ethnicity - social group or clutural tradition of a person

7.hosp_yn       - status of admission in hospitial 

8.icu_yn        - status of admission in ICU

9.death_yn      - did the patient died as a result of this illness

10.medcond_yn   - status of pre-existing medical condition

In [ ]:
#data type of various columns
df.info()

In [ ]:
#Changing object to datetime 
df['cdc_report_dt']=pd.to_datetime(df['cdc_report_dt'])

df['onset_dt']=pd.to_datetime(df.onset_dt)
df['pos_spec_dt']=pd.to_datetime(df.pos_spec_dt)


In [ ]:
#checking the datatypes
df.info()

In [ ]:
#After changing the columns datatype let's check for missing in the dataset
df.isnull().sum()

In [ ]:
#Percentage of missing values in each columns would be considered better for understanding

def percentage_of_missing():
    t=df.columns 
    for i in t:
        t=(df[i].isnull().sum())/df.shape[0]
        print(i[:4],'\t\t',round(t*100,4),'%')
percentage_of_missing()

In [ ]:
msno.matrix(df)


In [ ]:
#Heatmap with msno helps to know how strongly the presence or absence of one variable affects the presence of another
msno.heatmap(df)

In [ ]:
 msno.dendrogram(df)

 # DataPreprocessing

In [ ]:

#Instead of deleting the columns pos_spec_dt and onset_pos_difference we will try to fill the missing values by taking the difference between them.
df['onset_pos_difference'] = (df['onset_dt'] -df['pos_spec_dt'][df.pos_spec_dt.notnull()]).dt.days
df['pos_difference'] = (df['cdc_report_dt'] -df['pos_spec_dt'][df.pos_spec_dt.notnull()]).dt.days
df['onset_difference'] = (df['cdc_report_dt'] -df['onset_dt'][df.onset_dt.notnull()]).dt.days

In [ ]:
print('on_pos_set:-',df['onset_pos_difference'].mean(),df['onset_pos_difference'].median(),df['onset_pos_difference'].mode())

print('onset_set:-',df['onset_difference'].mean(),df['onset_difference'].median(),df['onset_difference'].mode())

print('pos_difference:-',df['pos_difference'].mean(),df['pos_difference'].mode(),df['pos_difference'].median())


In [ ]:
df['onset_dt'].fillna(df.cdc_report_dt + timedelta(days=2), inplace = True)

df['pos_spec_dt'].fillna(df.cdc_report_dt+timedelta(days=2),inplace=True)

In [ ]:
#Extracting month 
df['month_cdc']=df['cdc_report_dt'].dt.month
#Extracting week
df['week_cdc']=df['cdc_report_dt'].dt.week

In [ ]:
df.drop(columns=['onset_pos_difference','onset_difference','pos_difference'],inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.sex.unique()

In [ ]:
df['sex'].value_counts()

In [ ]:
df[df['age_group'].isna()][:29]

In [ ]:
df['sex']=df.groupby(['death_yn','icu_yn','month_cdc']).sex.transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
df['sex'].unique()

In [ ]:
df['sex'].value_counts()

In [ ]:
df['sex'].replace({'Missing':np.nan,'Unknown':np.nan},inplace=True)

In [ ]:
df['sex']=df.groupby(['death_yn','icu_yn','month_cdc']).sex.transform(lambda x: x.fillna(x.mode()[0]))

df['sex'].unique()

In [ ]:
#filling missing values in Race and ethnicity
df['Race and ethnicity (combined)'].unique()

# Apart from missing values there were Unkown, missing separately
How many missing values are present ..?

How many Unkown values are present...?


In [ ]:
df['Race and ethnicity (combined)'].value_counts()

In [ ]:
#Replacing unkowns and missing to nan and then trying to fill the values 
df["Race and ethnicity (combined)"].replace({"Missing": np.nan, "Unknown": np.nan}, inplace=True)

In [ ]:
df['Race and ethnicity (combined)'].unique()

In [ ]:
df['Race and ethnicity (combined)']=df.groupby(['death_yn','icu_yn','month_cdc'])['Race and ethnicity (combined)'].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:

df['Race and ethnicity (combined)'].unique()

In [ ]:
#Missing values in age_group column

df['age_group'].unique()

In [ ]:
df['age_group'].replace({'Unknown':np.nan},inplace=True)

In [ ]:
df['age_group'].unique()

In [ ]:
df['age_group']=df.groupby(['death_yn','icu_yn','month_cdc','Race and ethnicity (combined)'])['age_group'].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
df['age_group'].unique()

In [ ]:
#checking in other columns  for missing and Unkown data

df['hosp_yn'].unique()

In [ ]:
df['hosp_yn'].replace({'Missing':np.nan,'Unknown':np.nan},inplace=True)
df['hosp_yn'].unique()


In [ ]:
df['hosp_yn']=df.groupby(['death_yn','Race and ethnicity (combined)'])['hosp_yn'].transform(lambda x: x.fillna(x.mode()[0]))
df['hosp_yn'].unique()

In [ ]:
df['hosp_yn'].unique()

In [ ]:
#Checking icu_yn
df['icu_yn'].unique()

In [ ]:
df['icu_yn'].replace({'Missing':np.nan,'Unknown':np.nan},inplace=True)

In [ ]:
df['icu_yn']=df.groupby(['age_group','Race and ethnicity (combined)'])['icu_yn'].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
df['icu_yn'].unique()

In [ ]:
#death column
df['death_yn'].unique()

In [ ]:
df['death_yn'].replace({'Missing':np.nan,'Unknown':np.nan},inplace=True)

In [ ]:
df['death_yn'].unique()

In [ ]:
df['death_yn']=df.groupby(['age_group','icu_yn','Race and ethnicity (combined)'])['death_yn'].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
df['death_yn'].unique()

In [ ]:
#medcond_yn column
df['medcond_yn'].unique()

In [ ]:
df['medcond_yn'].replace({'Missing':np.nan},inplace=True)

In [ ]:
df['medcond_yn']=df.groupby(['icu_yn','Race and ethnicity (combined)'])['medcond_yn'].transform(lambda x: x.fillna(x.mode()[0]))


In [ ]:
df['medcond_yn'].unique()

In [ ]:
#checking current status column
df['current_status'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

In [ ]:
df['death_yn'].unique()

In [ ]:
df['death_yn'].replace({'Yes':1,'No':0},inplace=True)

# DataVisualization 

# How to know the values distribution of age group ?

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(df['age_group'],log=True)
plt.xlabel("Age group")
plt.xticks(rotation=45)
plt.show()

# How to know the values distribution of Race and ethnicity..?

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(df['Race and ethnicity (combined)'],log=True)
plt.xlabel("Race Ethnicity")
plt.xticks(rotation=45)
plt.show()

# How to know the values distribution in Medical condition..?

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(df['medcond_yn'],log=True)
plt.xlabel("Medical condition")
plt.xticks(rotation=45)
plt.show()

# Spread of admission based on gender in ICU..?


In [ ]:

plt.figure(figsize=(10,7))
sns.histplot(data=df, x="icu_yn", hue="sex", multiple="stack")
plt.xlabel("Admission's in ICU")
plt.show()

# How to know the current status of people based on age group..?

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(y="current_status",hue ='age_group',data=df)
plt.show()

# How to know the death of people based on age group..?


In [ ]:

plt.figure(figsize=(8,6))
sns.countplot(y='death_yn',hue='age_group',data=df)
plt.ylabel('death')
plt.show()

# How to know to count of people joined in icu based on age group..?

In [ ]:

plt.figure(figsize=(8,6))
sns.countplot(y='icu_yn',hue='age_group',data=df)
plt.ylabel('admission in icu')
plt.show()

# How to know the death of people based on race and ethnicity..?

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(y='death_yn',hue='Race and ethnicity (combined)',data=df)
plt.ylabel('Death')
plt.show()

# How to know the count of people based on age group and gender..?


In [ ]:
plt.figure(figsize=(15,7))
sns.histplot(data=df,x='age_group',bins=30,hue='sex',multiple='stack')
plt.xlabel("Age group")
plt.title(" Count of age group's")
plt.show()

# How to figureout the  count of people joined in icu based on age groups..?

In [ ]:
plt.figure(figsize=(15,7))
sns.histplot(data=df,x='age_group',bins=30,hue='icu_yn',multiple='stack')
plt.xlabel("Age group")
plt.title(" Count of age group's")
plt.show()

In [ ]:
df['death_yn'].unique()

In [ ]:
values = df['death_yn'].value_counts().tolist()
names = [ 'Yes', 'No']
fig = px.pie(names=names,values=values, title="Distribution Pre-Existing Medical Conditions",)
fig.show()


In [ ]:
values = df['medcond_yn'].value_counts().tolist()
names = [ 'Unknown', 'Yes', 'No']
fig = px.pie(names=names,values=values, title="Distribution Pre-Existing Medical Conditions",)
fig.show()


In [ ]:
values = df['current_status'].value_counts().tolist()
names = ['Confirmed', 'Probable']
px.pie( names=names,    values=values,title="Case Status Distribution")



In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.pairplot(df)
plt.show()

In [ ]:
#df['year']
df['cdc_report_dt'].dt.year.max(),df['cdc_report_dt'].dt.year.min()#as the year is same so not considering year column

In [ ]:
data=df.groupby(["week_cdc",'death_yn']).count()
data.reset_index(inplace=True)

# How to know the trend of deaths based on weeks of a year..?

In [ ]:
plt.figure(figsize=(10,7))
sns.lineplot(data=data,x='week_cdc',y='cdc_report_dt')
plt.show()

#  How to know the trend of deaths based on weeks of a year , age group , admissions in icu and death of a person..?

In [ ]:
data=df.groupby(["week_cdc",'age_group','icu_yn','death_yn']).count()
data.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
sns.lineplot(data=data,x='week_cdc',y='cdc_report_dt')
plt.show()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.current_status.unique()

In [ ]:
df.sex.unique()

In [ ]:
df.age_group.unique()

In [ ]:
df['Race and ethnicity (combined)'].unique()

In [ ]:
df.hosp_yn.unique()

In [ ]:
df.icu_yn.unique()

In [ ]:
df.medcond_yn.unique()

In [ ]:
df['month_cdc'].unique()

In [ ]:
df.current_status.replace({'Laboratory-confirmed case':1,'Probable Case':2},inplace=True)

In [ ]:
df.sex.replace({'Male':1,'Female':2,'Other':3},inplace=True)


In [ ]:
df.age_group.replace({'10 - 19 Years':1, '20 - 29 Years':2, '30 - 39 Years':2, '40 - 49 Years':3,'50 - 59 Years':3, '0 - 9 Years':1, '60 - 69 Years':4, '80+ Years':4,'70 - 79 Years':4},inplace=True)

In [ ]:
df['Race and ethnicity (combined)'].replace({'Black, Non-Hispanic':1, 'White, Non-Hispanic':2,
       'Asian, Non-Hispanic':3,
       'American Indian/Alaska Native, Non-Hispanic':4,
       'Multiple/Other, Non-Hispanic':5,
       'Native Hawaiian/Other Pacific Islander, Non-Hispanic':6,
       'Hispanic/Latino':7},inplace=True)

In [ ]:
df['hosp_yn'].replace({'No':0,'Yes':1},inplace=True)

In [ ]:
df['icu_yn'].replace({'No':0,'Yes':1},inplace=True)

In [ ]:
df['medcond_yn'].replace({'No':0, 'Yes':1, 'Unknown':2},inplace=True)

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(),annot=True,annot_kws={'size':9})

# Implementing machine learning algorthm

# Performing classification problem based on Death condition


In [ ]:
X=df.drop(columns=['death_yn','month_cdc','week_cdc','Race and ethnicity (combined)','age_group','pos_spec_dt','onset_dt','cdc_report_dt'])

In [ ]:
y=df['death_yn']


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier  


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#Logistic Regression
logistic_reg = LogisticRegression()
pred=logistic_reg.fit(X_train, y_train)
#Predicting values
Y_pred = logistic_reg.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred))

In [ ]:
confusion_matrix(y_test,Y_pred)

In [ ]:
plot_confusion_matrix(logistic_reg, X_test, y_test)  

In [ ]:
#RandomForest

rf_c=RandomForestClassifier()
pred_rf=rf_c.fit(X_train, y_train)
#Predicting values
Y_pred_rf = pred_rf.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred_rf))

In [ ]:
confusion_matrix(y_test,Y_pred_rf)

In [ ]:
plot_confusion_matrix(rf_c, X_test, y_test)  

In [ ]:
#Naive Bayes
Nb=GaussianNB()
pred_nb=Nb.fit(X_train, y_train)
#Predicting values
Y_pred_nb = pred_nb.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred_nb))

In [ ]:
confusion_matrix(y_test,Y_pred_nb)

In [ ]:
plot_confusion_matrix(Nb, X_test, y_test)

In [ ]:
#Decision tree classifier
DC= DecisionTreeClassifier()  
pred_dc=DC.fit(X_train, y_train)

In [ ]:
Y_pred_dc= pred_dc.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred_dc))

In [ ]:
confusion_matrix(y_test,Y_pred_dc)

In [ ]:
plot_confusion_matrix(DC, X_test, y_test)

# Performing classification problem based on joined in icu 

In [ ]:
X=df.drop(columns=['death_yn','month_cdc','week_cdc','Race and ethnicity (combined)','age_group','pos_spec_dt','onset_dt','cdc_report_dt','current_status','icu_yn'])

In [ ]:
y=df['icu_yn']

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#Logistic Regression
logistic_reg = LogisticRegression()
pred=logistic_reg.fit(X_train, y_train)
#Predicting values
Y_pred = logistic_reg.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred))

In [ ]:
confusion_matrix(y_test,Y_pred)

In [ ]:
plot_confusion_matrix(logistic_reg, X_test, y_test)

In [ ]:
#RandomForest

rf_c=RandomForestClassifier()
pred_rf=rf_c.fit(X_train, y_train)
#Predicting values
Y_pred_rf = pred_rf.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred_rf))

In [ ]:
confusion_matrix(y_test,Y_pred_rf)

In [ ]:
plot_confusion_matrix(rf_c, X_test, y_test)  

In [ ]:
#Naive Bayes
Nb=GaussianNB()
pred_nb=Nb.fit(X_train, y_train)
#Predicting values
Y_pred_nb = pred_nb.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred_nb))

In [ ]:
confusion_matrix(y_test,Y_pred_nb)

In [ ]:
plot_confusion_matrix(Nb, X_test, y_test)

In [ ]:
#Decision tree classifier
DC= DecisionTreeClassifier()  
pred_dc=DC.fit(X_train, y_train)

In [ ]:
Y_pred_dc= pred_dc.predict(X_test)

In [ ]:
print(classification_report(y_test, Y_pred_dc))

In [ ]:
confusion_matrix(y_test,Y_pred_dc)

In [ ]:
plot_confusion_matrix(DC, X_test, y_test)